# This notebook creates the MIRI Imager boresight offsets table for the PRD #

In [1]:
import numpy as np
import os
import getpass
import socket
import pdb as pdb
from astropy.modeling import models
from asdf import AsdfFile
from jwst import datamodels
from jwst.assign_wcs import miri
from astropy.io import fits
import datetime

Import the MIRI coordinates code from https://github.com/STScI-MIRI/miricoord and ensure that it is on the PYTHONPATH.  Also ensure that the output data directory is set:<br>
setenv MIRICOORD_DATA_DIR /YourLocalPathToData/ (this is where io will happen)

In [2]:
data_dir=os.path.expandvars('$MIRICOORD_DATA_DIR')

Import the MIRI standalone tools

In [3]:
import miricoord.miricoord.imager.mirim_tools as mt

Import the MIRI pipeline tools

In [4]:
import miricoord.miricoord.imager.mirim_pipetools as mpt

<br>

Import the pysiaf tools for working with the SIAF, and read the MIRI apertures

In [5]:
import pysiaf

In [6]:
siaf = pysiaf.Siaf('MIRI') 

In [7]:
siafversion=pysiaf.JWST_PRD_VERSION

<BR>

Load up the MIRI Imager distortion model (using F770W since we're COMPUTING offsets rather than using them)

In [8]:
model=mpt.xytov2v3model('F770W')

<BR>

Point to the IDT-delivered CDP file that we're using

In [9]:
cdpfile=mt.get_fitsreffile()

In [10]:
hdul = fits.open(cdpfile)

In [11]:
tbdata = hdul['Boresight offsets'].data

In [12]:
#print(tbdata.columns)

In [13]:
allfilters=tbdata.field('filter')

In [14]:
nfilter=len(allfilters)

In [15]:
#print(tbdata)

<BR>

Define two output files: One for TA boresights, one for science boresights

In [16]:
taoutfile=data_dir+'boresight/temp/miri_boresight_ta.txt'
scioutfile=data_dir+'boresight/temp/miri_boresight_sci.txt'

Log information about when the files were run, with what inputs

In [17]:
now=datetime.datetime.now()

In [18]:
print('#',now.isoformat(),file=open(taoutfile,"w"))
print('#',now.isoformat(),file=open(scioutfile,"w"))
print('# Created by user',getpass.getuser(),'at',socket.gethostname(),file=open(taoutfile,"a"))
print('# Created by user',getpass.getuser(),'at',socket.gethostname(),file=open(scioutfile,"a"))
print('# Boresight pixel file:',cdpfile,file=open(taoutfile,"a"))
print('# Boresight pixel file:',cdpfile,file=open(scioutfile,"a"))
print('# SIAF file:',siafversion,file=open(taoutfile,"a"))
print('# SIAF file:',siafversion,file=open(scioutfile,"a"))
print('# Offsets (arcsec) are set so that they should be ADDED to the commanded position',file=open(taoutfile,"a"))
print('# Offsets (arcsec) are set so that they should be ADDED to the commanded position',file=open(scioutfile,"a"))

Print a header line about the column entries

In [19]:
print("{:20} {:10} {:10} {:10}".format('# AperName','TAFilter','dv2','dv3'), file=open(taoutfile,"a"))
print("{:20} {:10} {:10} {:10}".format('# AperName','SciFilter','dv2','dv3'), file=open(scioutfile,"a"))

<BR>

Set up the desired sign of the outputs.  We want APT to ADD both the TA filter and science filter corrections, therefore we need to incorporate the negative sign on the TA filter offset into our reported values

In [20]:
tasign=-1
scisign=+1

<BR>

We will in each case do the conversion from pixel boresight offset to v2/v3 offset.  We will use the location of the TA aperture for MRS and LRS slit (because these science apertures don't have explicit pixel locations on the imager), and the science aperture for everything else.  Offsets are given in v2/v3 to avoid ambiguities with the Ideal coordinate systems.

<BR>

## Do the MRS case ##

In [21]:
print("",file=open(taoutfile,"a"))
print("",file=open(scioutfile,"a"))

# Loop over each TA filter computing offsets
for i in range(0,nfilter):
    thisentry=siaf['MIRIM_TAMRS']
    xdet,ydet=thisentry.XDetRef-1,thisentry.YDetRef-1 # Define the location in 0-indexed pixels (subtract 1 from SIAF)
    v2ref,v3ref=thisentry.V2Ref,thisentry.V3Ref
    x,y=xdet+tbdata[i].field('col_offset'),ydet+tbdata[i].field('row_offset')
    v2,v3=model(x,y)
    dv2,dv3=v2-v2ref,v3-v3ref
    # Print the negative of the offsets so that they can be added to the commanded position
    print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(tasign*dv2,tasign*dv3,val1='MIRIFU*',val2=tbdata[i].field('filter')),file=open(taoutfile,"a"))
    
# Loop over each science filter (for MRS only one option)
print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(0,0,val1='MIRIFU*',val2='ALL'),file=open(scioutfile,"a"))

<BR>

## Do the LRS slit case ##

In [22]:
print("",file=open(taoutfile,"a"))
print("",file=open(scioutfile,"a"))

# Loop over each TA filter computing offsets AT THE SCIENCE APERTURE LOCATION
for i in range(0,nfilter):
    thisentry=siaf['MIRIM_TALRS']
    xdet,ydet=thisentry.XDetRef-1,thisentry.YDetRef-1 # Define the location in 0-indexed pixels (subtract 1 from SIAF)
    v2ref,v3ref=thisentry.V2Ref,thisentry.V3Ref
    x,y=xdet+tbdata[i].field('col_offset'),ydet+tbdata[i].field('row_offset')
    v2,v3=model(x,y)
    dv2,dv3=v2-v2ref,v3-v3ref
    # Print the value of the offsets so that they can be added to the commanded position
    print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(tasign*dv2,tasign*dv3,val1='MIRIM_SLIT',val2=tbdata[i].field('filter')),file=open(taoutfile,"a"))
    
# Loop over each science filter (for LRS only one option)
print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(0,0,val1='MIRIM_SLIT',val2='ALL'),file=open(scioutfile,"a"))

<br>

## Do the LRS slitless case ##

In [23]:
print("",file=open(taoutfile,"a"))
print("",file=open(scioutfile,"a"))

# Loop over each TA filter computing offsets
for i in range(0,nfilter):
    thisentry=siaf['MIRIM_SLITLESSPRISM']
    xdet,ydet=thisentry.XDetRef-1,thisentry.YDetRef-1 # Define the location in 0-indexed pixels (subtract 1 from SIAF)
    v2ref,v3ref=thisentry.V2Ref,thisentry.V3Ref
    x,y=xdet+tbdata[i].field('col_offset'),ydet+tbdata[i].field('row_offset')
    v2,v3=model(x,y)
    dv2,dv3=v2-v2ref,v3-v3ref
    # Print the value of the offsets so that they can be added to the commanded position
    print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(tasign*dv2,tasign*dv3,val1='MIRIM_SLITLESSPRISM',val2=tbdata[i].field('filter')),file=open(taoutfile,"a"))
    
# Loop over each science filter (for LRS only one option)
print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(0,0,val1='MIRIM_SLITLESSPRISM',val2='ALL'),file=open(scioutfile,"a"))

<BR>

## Do the coronagraphs ##

Note that we use the 'MASK' apertures since these are the ones that drive the actual pointing; the 'CORON' apertures are just convenient for display purposes

In [24]:
apernames=['MIRIM_MASK1065','MIRIM_MASK1140','MIRIM_MASK1550','MIRIM_MASKLYOT']
naper=len(apernames)

In [25]:
# Loop over each aperture
for j in range(0,naper):
    print("",file=open(taoutfile,"a"))
    print("",file=open(scioutfile,"a"))

    # Loop over each TA filter computing offsets
    for i in range(0,nfilter):
        thisentry=siaf[apernames[j]]
        xdet,ydet=thisentry.XDetRef-1,thisentry.YDetRef-1 # Define the location in 0-indexed pixels (subtract 1 from SIAF)
        v2ref,v3ref=thisentry.V2Ref,thisentry.V3Ref
        x,y=xdet+tbdata[i].field('col_offset'),ydet+tbdata[i].field('row_offset')
        v2,v3=model(x,y)
        dv2,dv3=v2-v2ref,v3-v3ref
        # Print the value of the offsets so that they can be added to the commanded position
        print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(tasign*dv2,tasign*dv3,val1=apernames[j],val2=tbdata[i].field('filter')),file=open(taoutfile,"a"))
    
    # Loop over each science filter computing offsets
    for i in range(0,nfilter):
        thisentry=siaf[apernames[j]]
        xdet,ydet=thisentry.XDetRef-1,thisentry.YDetRef-1 # Define the location in 0-indexed pixels (subtract 1 from SIAF)
        v2ref,v3ref=thisentry.V2Ref,thisentry.V3Ref
        x,y=xdet+tbdata[i].field('col_offset'),ydet+tbdata[i].field('row_offset')
        v2,v3=model(x,y)
        dv2,dv3=v2-v2ref,v3-v3ref
        # Print the value of the offsets so that they can be added to the commanded position
        print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(scisign*dv2,scisign*dv3,val1=apernames[j],val2=tbdata[i].field('filter')),file=open(scioutfile,"a"))
    

<BR>

## Do the imaging regions ## 

In [26]:
apernames=['MIRIM_FULL','MIRIM_ILLUM','MIRIM_BRIGHTSKY','MIRIM_SUB256','MIRIM_SUB128','MIRIM_SUB64']
naper=len(apernames)

In [27]:
# Loop over each aperture
for j in range(0,naper):
    print("",file=open(taoutfile,"a"))
    print("",file=open(scioutfile,"a"))

    # Loop over each TA filter computing offsets
    for i in range(0,nfilter):
        thisentry=siaf[apernames[j]]
        xdet,ydet=thisentry.XDetRef-1,thisentry.YDetRef-1 # Define the location in 0-indexed pixels (subtract 1 from SIAF)
        v2ref,v3ref=thisentry.V2Ref,thisentry.V3Ref
        x,y=xdet+tbdata[i].field('col_offset'),ydet+tbdata[i].field('row_offset')
        v2,v3=model(x,y)
        dv2,dv3=v2-v2ref,v3-v3ref
        # Print the value of the offsets so that they can be added to the commanded position
        print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(tasign*dv2,tasign*dv3,val1=apernames[j],val2=tbdata[i].field('filter')),file=open(taoutfile,"a"))
    
    # Loop over each science filter computing offsets
    for i in range(0,nfilter):
        thisentry=siaf[apernames[j]]
        xdet,ydet=thisentry.XDetRef-1,thisentry.YDetRef-1 # Define the location in 0-indexed pixels (subtract 1 from SIAF)
        v2ref,v3ref=thisentry.V2Ref,thisentry.V3Ref
        x,y=xdet+tbdata[i].field('col_offset'),ydet+tbdata[i].field('row_offset')
        v2,v3=model(x,y)
        dv2,dv3=v2-v2ref,v3-v3ref
        # Print the value of the offsets so that they can be added to the commanded position
        print("{val1:20} {val2:10} {0:<10.4f} {1:<10.4f}".format(scisign*dv2,scisign*dv3,val1=apernames[j],val2=tbdata[i].field('filter')),file=open(scioutfile,"a"))
    